In [62]:
import tensorflow as tf
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
import pandas as pd
import numpy as np

from spacy.lang.en import English
from nltk.corpus import stopwords
import nltk

In [63]:
# import model for testing
model_path = '../models/distilbert_model_best.h5'
model = TFDistilBertForSequenceClassification.from_pretrained(model_path)

# import tokenizer for testing
tokenizer_path = '../tokenizers/distilbert_tokenizer_best'
tokenizer = DistilBertTokenizer.from_pretrained(tokenizer_path)


Some layers from the model checkpoint at ../models/distilbert_model_best.h5 were not used when initializing TFDistilBertForSequenceClassification: ['dropout_199']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at ../models/distilbert_model_best.h5 and are newly initialized: ['dropout_99']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [134]:
# Download the dictionary for stopwords
nltk.download('stopwords')
# Get the set of stopwords
stop_words_set = set(stopwords.words('english'))
# Load English tokenizer from spacy
nlp = English()
spacy_tokenizer = nlp.tokenizer ## make instance
# Create function to clean text -- lowercase, remove non alphanumeric, remove stop words
def optimized_preprocess(text): ## Takes in a list of texts, i.e. the entire corpus
    # Tokenize using spaCy’s tokenizer
    tokens = [token.text.lower() for token in spacy_tokenizer(text) if token.text.isalpha() and token.text.lower() not in stop_words_set]
    cleaned_query= ' '.join(word for word in tokens)
    return cleaned_query

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/matthewieraci/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [65]:
# Load unseen dataset for testing
test_df = pd.read_csv('../data/train.csv')

# Seperate fake and true articles with random sample size variable
sample_size = 100
fake_df = test_df[test_df['label'] == 1].sample(sample_size)
true_df = test_df[test_df['label'] == 0].sample(sample_size)

In [66]:
# Display fake df
fake_df

,id,title,author,text,label
5600,5600,One Treaty Could Change the Fight to Stop the ...,Yessenia Funes,Tweet Widget by Yessenia Funes \nThe fight aga...,1
12478,12478,Pokemon GO Revelations/ Open Lines,NaN,Pokemon GO Revelations/ Open Lines Pokemon G...,1
14131,14131,11:11: 2016 – ‘The Lifting’,Gillian,Leave a reply \nGillian MacBeth Louthan – The ...,1
9712,9712,"She Disappeared Without A Trace In 1850, Then ...",Eddy Lavine,posted by Eddie Olive Oatman was born in Illin...,1
7685,7685,Pentagon Retreats on Enlistment Bonus Collecti...,NaN,Pentagon Retreats on Enlistment Bonus Collecti...,1
...,...,...,...,...,...
19462,19462,"Article 50 appeal to be heard by Mary Berry, M...",NewsBiscuit,"November 4, 2016 \nThe government has caused f...",1
15502,15502,Unbrexit! Parliament must vote on triggering a...,Sausage Machine,Next Prev Swipe left/right Unbrexit! Parliamen...,1
2253,2253,Jeff Gundlach’s Warning And Danger For Key Glo...,King World News,"40 Views October 26, 2016 GOLD , KWN King Worl...",1
9147,9147,Hillary’s Silent Plan to Destroy America,Dave Hodges,\n\nHillary’s secret agenda is not so secret. ...,1


In [138]:
# Display true df
true_df

,id,title,author,text,label
5155,5155,Former FEC Chairwoman Suggests Regulating Poli...,Lucas Nolan,Former chairwoman of the Federal Election Comm...,0
10631,10631,Jewish Center Bomb Threat Suspect Is Arrested ...,"Isabel Kershner, Adam Goldman, Alan Blinder an...",JERUSALEM — A monthslong wave of bomb threa...,0
13195,13195,What You Should Watch: ‘Greenleaf’ and ‘Thirte...,Margaret Lyons,"Welcome to Watching, The New York Times’s g...",0
13392,13392,Rubio: Many Humanitarian Questions About Trump...,Pam Key,"Sunday on ’s “Facing South Florida,” while di...",0
5503,5503,Trump’s Transportation Secretary Elaine Chao C...,Jerome Hudson,"Elaine Chao, Donald Trump’s nominee to head th...",0
...,...,...,...,...,...
4117,4117,Facebook to Appeal Court Ruling Imposing Austr...,Charlie Nash,Facebook is appealing an order to impose Austr...,0
2512,2512,Bannon Is Given Security Role Usually Held for...,Glenn Thrush and Maggie Haberman,WASHINGTON — The whirlwind first week of Do...,0
14510,14510,EXCLUSIVE -- Mexico Backs Down in Fight With U...,Ildefonso Ortiz and Brandon Darby,"REYNOSA, Tamaulipas — After erroneously tak...",0
18541,18541,Tensions Deepen Between Donald Trump and R.N.C...,Alexander Burns and Maggie Haberman,The Republican National Committee had high hop...,0


In [120]:
threshold = 0.7

In [137]:
# Function to test model on df of articles
# texts is the df being passed
# optimizer decides whether or not the articles will be preprocessed
# max_length decides size of articles to be passed into tokenizer
def test_model_df(texts, optimizer=True, max_length=300):
# Create empty list to store results
    check_list = []
    
    # Iterate through texts in the fake_df
    for text in texts.text:
        if optimizer == True:
        # store preprocessed text in random_input variable
            random_input = optimized_preprocess(text)
        else:
            random_input = text
        # tokenize the random input and return as a tensor
        random_input = tokenizer.encode_plus(
            random_input,
            add_special_tokens=True,
            max_length=max_length,
            truncation = True,
            padding='max_length', 
            return_attention_mask=True,
            return_tensors="tf"
        )
        
        # Extract 'input_ids' from the 'random_input' dictionary.
        # (numerical representations of textual input for the model)
        input_ids = random_input['input_ids']
        
        # Extract the 'attention_mask' from the 'random_input' dictionary. 
        # This mask helps the model focus on relevant parts of the input.
        attention_mask = random_input['attention_mask']

        # Pass the 'input_ids' and 'attention_mask' to the model to get predictions. 
        # The model uses these inputs to make predictions about the class of the input (Real or Fake).
        predictions = model(input_ids, attention_mask=attention_mask)
        
        # Apply the softmax function to the logits (raw outputs) of the model's predictions.
        # Softmax converts logits to probabilities, making them easier to interpret.
        probabilities = tf.nn.softmax(predictions.logits, axis=-1)


        max_probability = tf.reduce_max(probabilities, axis=-1).numpy()[0]

        if max_probability < threshold:
            classification = "Suspicious"
        else:
            predicted_class = tf.argmax(probabilities, axis=-1).numpy()
            class_names = ['Real', 'Fake']
            classification = class_names[predicted_class[0]]
            
        check_list.append([f"The article is predicted as: {classification}", f"Probabilities per class: {probabilities.numpy()[0]}"])
        
        if len(check_list) == texts.shape[0]:
            return check_list

In [143]:
# Run test with preprocessor
true_check_list_preproc = test_model_df(true_df)
fake_check_list_preproc = test_model_df(fake_df)

# Run test without preprocessor
true_check_list = test_model_df(true_df, optimizer=False)
fake_check_list = test_model_df(fake_df, optimizer=False)


In [149]:
# RUN THIS CELL ONLY ONCE

# Create an empty dict to store results (with preprocessing)
# Start at test #1
result_preproc_dict = {}
test_num_preproc = 1

# Create an empty dict to store results (without preprocessing)
# Start at test #1
result_dict = {}
test_num = 1

In [150]:
# Load test results in dict
result_preproc_dict[f'test_{test_num}'] = {
    # Sub dict for fake article test
    'fake_examples': {
        # Save indices of tested articles in list form
        'indices': fake_df.index.tolist(),
        # Save scores for test
        'scores': fake_check_list_preproc
    },
    # Sub dict for true article test
    'true_examples': {
        # Save indices of tested articles in list form
        'indices': true_df.index.tolist(),
        # Save scores for test
        'scores': true_check_list_preproc
    }
}
# Increase test_num by one to prepare for next test
test_num_preproc += 1

In [151]:
# Store test results
# refer to result dict cell block above for comments
result_dict[f'test_{test_num}'] = {
    'fake_examples': {
        'indices': fake_df.index.tolist(),
        'scores': fake_check_list
    },
    'true_examples': {
        'indices': true_df.index.tolist(),
        'scores': true_check_list
    }
}
test_num += 1

In [152]:
print(result_preproc_dict.keys(),result_dict.keys())

dict_keys(['test_1']) dict_keys(['test_1'])


In [167]:
# Calculate fake_df test accuracy
# ------WITH PREPROCESSING------
# Iterate through tests in the result_dict
def calc_accuracy(dict_):
    for test in dict_.keys():
        # Create a count variable to track results
        fake_count = 0
        true_count = 0
        fake_suspicious_count = 0
        true_suspicious_count = 0
        # Index into keys ('fake_examples', 'true_examples')
        for key in dict_[test]:
            # Index to list of scores
            for result in dict_[test][key]['scores']:
                # For score in list of scores, if wrongly predicted:
                for row in result:
                    if key == 'fake_examples':
                        if "Real" in row:
                            # Add one to count
                            fake_count += 1
                            # Print row that was incorrect
                            # print(result)
                        if "Suspicious" in row:
                            fake_suspicious_count += 1
                            # print(result)
                    if key == 'true_examples':
                        if "Fake" in row:
                            # Add one to count
                            true_count += 1
                            # Print row that was incorrect
                            # print(result)
                        if "Suspicious" in row:
                            true_suspicious_count += 1
                            # print(result)
    # Print the accuracy for the test in question in percentage format
    if dict_ == result_preproc_dict:
        return [
            [f'Accuracy (Fake W/ Preprocessing) = {100 - fake_count}% Percentage of Suspicious Classification = {fake_suspicious_count}%'],
            [f'Accuracy (True W/ Preprocessing) = {100 - true_count}% Percentage of Suspicious Classification = {true_suspicious_count}%']
            ]
    else:
        return [
            [f'Accuracy (Fake NO Preprocessing) = {100 - fake_count}% Percentage of Suspicious Classification = {fake_suspicious_count}%'],
            [f'Accuracy (True NO Preprocessing) = {100 - true_count}% Percentage of Suspicious Classification = {true_suspicious_count}%']
            ]

In [170]:
for list in calc_accuracy(result_dict):
    print(list)
    
for list in calc_accuracy(result_preproc_dict):
    print(list)

['Accuracy (Fake NO Preprocessing) = 91% Percentage of Suspicious Classification = 22%']
['Accuracy (True NO Preprocessing) = 100% Percentage of Suspicious Classification = 6%']
['Accuracy (Fake W/ Preprocessing) = 93% Percentage of Suspicious Classification = 7%']
['Accuracy (True W/ Preprocessing) = 97% Percentage of Suspicious Classification = 6%']


In [174]:
# Function to test model on df of articles
# texts is the df being passed
# optimizer decides whether or not the articles will be preprocessed
# max_length decides size of articles to be passed into tokenizer
def test_article(article, optimizer=True, max_length=300):

    if optimizer == True:
    # store preprocessed text in random_input variable
        random_input = optimized_preprocess(article)
    else:
        random_input = article
    # tokenize the random input and return as a tensor
    random_input = tokenizer.encode_plus(
        random_input,
        add_special_tokens=True,
        max_length=max_length,
        truncation = True,
        padding='max_length', 
        return_attention_mask=True,
        return_tensors="tf"
    )
    
    # Extract 'input_ids' from the 'random_input' dictionary.
    # (numerical representations of textual input for the model)
    input_ids = random_input['input_ids']
    
    # Extract the 'attention_mask' from the 'random_input' dictionary. 
    # This mask helps the model focus on relevant parts of the input.
    attention_mask = random_input['attention_mask']

    # Pass the 'input_ids' and 'attention_mask' to the model to get predictions. 
    # The model uses these inputs to make predictions about the class of the input (Real or Fake).
    predictions = model(input_ids, attention_mask=attention_mask)
    
    # Apply the softmax function to the logits (raw outputs) of the model's predictions.
    # Softmax converts logits to probabilities, making them easier to interpret.
    probabilities = tf.nn.softmax(predictions.logits, axis=-1)


    max_probability = tf.reduce_max(probabilities, axis=-1).numpy()[0]

    if max_probability < threshold:
        classification = "Suspicious"
    else:
        predicted_class = tf.argmax(probabilities, axis=-1).numpy()
        class_names = ['Real', 'Fake']
        classification = class_names[predicted_class[0]]
        
    return [f"The article is predicted as: {classification}", f"Probabilities per class: {probabilities.numpy()[0]}"]

In [189]:
montreal_gazette_1 = '''
The Montreal police say they have solved a series of break-ins at businesses in the West Island following the arrest last week of a 33-year-old man with a lengthy criminal record.

“Justin Charland allegedly smashed the windows of around 10 businesses in order to steal cash or valuables,” the Service de police de la Ville de Montréal wrote in a release on Wednesday. “The suspect allegedly acted mainly at night while the targeted establishments were closed. Taking advantage of the absence of witnesses, he allegedly threw rocks into shop windows to shatter them. The investigation team believes he used this same procedure 10 times in as many boutiques and stores.”

Article content
Article content
The break-ins caused heavy damage to the businesses that were targeted. The businesses were located in areas covered by neighbourhood police Station 1, which includes municipalities on the western part of the island like Kirkland and Beaconsfield, and Station 3, which covers Île Bizard, Roxboro and Pierrefonds.

“The two neighbourhood stations worked closely with investigators in order to trace the suspect as quickly as possible,” the Montreal police wrote. “These joint efforts ultimately bore fruit and led to the arrest of Justin Charland. The latter was arrested on Dec. 7 while he was at his home in Kirkland. Evidence and stolen items were seized during the subsequent search.

“The investigation is continuing to determine whether Charland could be linked to other similar events that occurred in the same area.”

Last week, Charland was charged with three counts of breaking and entering. He was scheduled to have a bail hearing on Wednesday at the Montreal courthouse.

Charland was on probation when he was arrested. On June 1, 2022, he pleaded guilty to breaking and entering and was sentenced in March to a six-month prison term to be followed by two years of probation.
'''
montreal_gazette_2 = '''
The Legault government is reportedly ready to significantly increase university tuition for students from other Canadian provinces — and lower rates for francophone students from France, Belgium, and maybe Switzerland.

Quebec on Wednesday was also mulling over a possible new measure: requiring 80 per cent of international students to reach intermediate French proficiency by graduation, according to La Presse.

Article content
Article content
McGill and Concordia universities say the tuition hike and the French-proficiency rules would be disastrous, driving away many students from other provinces and countries.

Citing unnamed sources, La Presse reported on what Premier François Legault’s cabinet is considering behind closed doors.

Most students from other provinces who come to Quebec to study do so at one of three English universities: Bishop’s, Concordia or McGill.

The government’s initial plan, announced in October, would have almost doubled the out-of-province rate, to $17,000, as of fall 2024. Now, Quebec is reportedly ready to set it at $12,000. That’s a 33-per-cent increase compared to current rates.

English universities and Quebec’s largest employer’s group, the Conseil du patronat, have said this will still be too high.

They say a major tuition hike will scare off most students from other provinces, leading to a catastrophic drop in revenue. Concordia and McGill have already imposed hiring freezes and warn they may have to cut jobs. Bishop’s, Quebec’s smallest English university, has said it might not survive.

Montreal Gazette Headline News
Headline News
Get the latest headlines, breaking news and columns.

youremail@email.com
Sign Up
By signing up you consent to receive the above newsletter from Postmedia Network Inc.

Article content
Article content
Hoping to avert the tuition increase, Bishop’s, Concordia and McGill have offered to improve the teaching of French to students from the rest of Canada.

Under their offer, at least 40 per cent of non-French-speaking students enrolled in undergraduate programs would reach intermediate proficiency in French by the time they graduate. The English universities said their francization efforts would be an alternative to the tuition hike.

Instead, the government plans to do both — increase tuition while obliging the universities to francize students, La Presse reported.

And, instead of 40 per cent, the province is reportedly considering an 80-per-cent target. The universities could face penalties if the target, to be in place by fall 2025, isn’t met.

This would be “disastrous” because many international students “have zero notion of French,” Fabrice Labeau, McGill’s deputy provost of student life and learning, told the Montreal Gazette.

For such a student to reach intermediate proficiency would require “240 hours of courses or something like 18 university credits – that’s more than a normal semester.”

Article content
Labeau cited as an example a German student considering McGill vs. the University of Toronto. They would likely balk at McGill because of the extra semester, he said.

“The fact that we speak French in Montreal used to be an asset for recruitment,” he said. “We’re making it a punishment now.”

He added: “We’re slamming the door on the Canadian students (with the tuition hike). Now, if we add on top of that, the French requirement, we’re slamming the door on international students.”

Concordia spokesperson Fiona Downey said an 80-per-cent French target is “inconceivable and unattainable.” That university has seen a 32-per-cent drop in applications from international students, compared to the same period last year.

In Quebec City, Higher Education Minister Pascale Déry told reporters the government is “going to come with an update very, very shortly.”

Government officials later told the Gazette that an announcement was expected on Thursday.

Déry was asked about complaints from McGill and Concordia that the government has given them the cold shoulder. Both institutions said Tuesday they haven’t been able to reach Déry since late November.

Article content
“We had a couple of discussions and I did speak to them obviously in those discussions,” Déry responded.

La Presse reported Quebec may cut tuition by two-thirds for university students from France and Belgium. They already pay much less than other international students when they study in Quebec.

The province also wants to negotiate a deal that would give francophone students from Switzerland the same tuition discount.

Students from France and Belgium currently pay the same tuition as those from the rest of Canada — $8,992 — when studying for bachelor’s degrees in Quebec. Students from Quebec pay $2,881.

Under the new scenario, French and Belgian students would pay $2,881, bringing the rate back down to where it was before 2015.

Just over 16,000 students from the two countries were studying in Quebec in 2021, a recent government report said. The vast majority are from France. Quebec had a total of 48,400 international students in 2021.

Legault has indicated he may exempt Bishop’s from the tuition hike. That university will have “no comment until we get the official announcement from the minister,” a Bishop’s spokesperson said.

Article content
A coalition of 40 anglophone groups denounced the tuition and French-proficiency changes.

“If a government were trying to devise a plan to starve Quebec’s English-language universities out of existence, it would look a lot like this,” said Eva Ludvig, president of the Quebec Community Groups Network.

“McGill is a jewel in Quebec’s crown — one of the world’s leading universities and consistently the best in Canada,” she said. “Instead of celebrating it and urging it on to greater success, the Legault government wants to punish it — all in the name of protecting and promoting the French language against what it perceives to be a threat in downtown Montreal.”

RELATED STORIES

Premier François Legault and Higher Education Minister Pascale Déry attend an event at the HEC Montréal business school on Friday, Sept. 15, 2023. Concordia and McGill say they have not spoken to Déry since late November.
Legault is giving English universities the cold shoulder: McGill and Concordia
McGill University principal Deep Saini, left, and Fabrice Labeau, the school's deputy provost of student life and learning, at a media briefing in Montreal Thursday December 7, 2023.
McGill won’t rule out moving some operations to another province

Legault has said the tuition hike will protect the French language and boost French universities. He has said non-French-speaking students from the rest of Canada are anglicizing Montreal.

But the QCGN argues the plan will not add revenue to the French university system. And it says the government has produced “no evidence whatsoever that the presence of McGill and Concordia students in the downtown core contributes to the anglicization of the metropolis.”

ariga@postmedia.com
'''
montreal_gazette_3 = '''
QUEBEC — The Legault government will study the finances of Montreal’s Olympic Park in January, and it won’t be until then that the cost of any new roof for the stadium will be revealed, Quebec Tourism Minister Caroline Proulx told reporters on Wednesday.

Arriving at the weekly cabinet meeting, Proulx refused to confirm a report in La Presse that said Quebec taxpayers would have to foot a bill of at least $750 million to replace the Olympic Stadium roof.

Article content
Article content
“We can study it sometime in January, then we’ll get back to you with the cost, the schedules and all that involves a potential replacement of the roof,” Proulx said. “The cabinet will make its decision in January.”

Last October, Premier François Legault stated clearly that the Olympic Stadium needed a new roof and the venue needed a new lease on life, even though the operation might be costly.

The current roof has reached the end of its operational life. In 2017, the Liberal government of Philippe Couillard announced the roof would be replaced at a cost of $200 million to $250 million with delivery of the project expected in 2022. But in 2019, that timetable was pushed to 2024 because of the complexity of the project. Since then, the amended schedule has also been scrapped, and no new date has been provided.

On Wednesday, Proulx told reporters the stadium roof was torn in about 20,000 places, but she had no intention of “presuming” what cabinet would do.

“Doing roof work is never really the ‘sexiest’ thing,” she said. “(But) if we wait any longer, within a year or two, the Olympic Stadium will be completely closed.”

Montreal Gazette Headline News
Headline News
Get the latest headlines, breaking news and columns.

youremail@email.com
Sign Up
By signing up you consent to receive the above newsletter from Postmedia Network Inc.

Article content
Article content
On Tuesday, the Olympic Park announced the stadium would be closed until further notice to accommodate exploratory work necessary for any replacement of the roof, a decision that will force the CF Montréal soccer team to play its first matches of 2024 outside of its home stadium.

Meanwhile, cities like Toronto are able, with their infrastructure, to attract megastars like Taylor Swift, Proulx noted.

“Tourist spending associated with the presence of Taylor Swift in Toronto: US$1,600 per spectator, 42,000 spectators times five days, that’s $350 million that Quebec … is losing because currently, we’re not capable of having a stadium.”

According to the Minister of the Economy Pierre Fitzgibbon, responsible for Montreal, the solution is certainly not to “demolish” the stadium.

“It’s a structure that is still somewhat iconic for Montreal. If we don’t do anything with it, we will destroy it, which is not a good option for Montreal. … We have to do something,” he said.

“We are not demolishing the Olympic Stadium,” Montreal Mayor Valérie Plante agreed on Wednesday. A lot of money has been invested in the Olympic Stadium, it is also a symbol of Montreal. … The city has invested nearly $1.5 billion.

“It’s like Groundhog Day … so we can’t wait to find a lasting solution as quickly as possible,” she added.
'''
montreal_gazette_4 = '''
Premier François Legault said on Wednesday morning that contract talks with striking teachers unions are going “well, very well,” even as the two labour federations involved in those talks are saying they remain far from a settlement.

But while Legault said he has high hopes that “all the children will return to class next Monday,” negotiations in the health sector are “very, very difficult” and he doesn’t think an agreement will be reached before January.

Article content
Article content
The 95,000 teachers represented by the CSQ-affiliated Fédération des syndicats de l’enseignement (FSE) and the 66,000 represented by the Fédération autonome de l’enseignement (FAE) are on strike. The first group will end their latest series of strike days on Thursday while the latter are on unlimited strike.

The unions quickly contradicted Legault’s optimism.

“Contrary to what François Legault says, the news at the table is not encouraging. The government is blowing hot and cold. It promises us an opening and then immediately closes the door. The government’s strategy is clearly to divide the movement and exhaust the teachers,” the FAE replied.


Montreal Gazette Headline News
Headline News
Get the latest headlines, breaking news and columns.

youremail@email.com
Sign Up
By signing up you consent to receive the above newsletter from Postmedia Network Inc.

Article content
Article content
FSE president Josée Scalabrini had a similar response.

“I don’t know what the premier knows that I don’t know. I don’t know what (Treasury Board president Sonia) LeBel knows that I don’t know. But even if there are positive signs that negotiations are progressing quickly, yesterday there were still mandates missing and we were on both sides in an exploratory manner on the major needs. Are we about to settle? Not to my knowledge.”

She said it was unrealistic to expect an agreement by Monday morning.

“A negotiation on such delicate elements cannot be done in 48 hours, even very difficult in a week.”

But LeBel said “the possibility is there; it’s real” for a Monday return to class.

The FSE was due to return to the negotiating table on Wednesday afternoon.

Difficult negotiations in health
Three major health unions, the FIQ, the APTS and the FSSS, confirmed that negotiations are difficult.

Legault said Wednesday that there was “almost no openness to give us the flexibility that is absolutely necessary” in the health sector by the unions.

He did not specify which health unions he was referring to.

Article content
The Fédération interprofessionnelle de la santé (FIQ), which represents 80,000 nurses and other health-care workers, painted a dark picture.

“Work is progressing at a snail’s pace at the negotiating table; the management side is not at all receptive,” said negotiating committee member Stella Larochelle in a video addressed to members.

Flexibility is a key issue. The FIQ says the government wants to move its members between care units or even establishments, which it says reflects a lack of recognition of nurses’ experience.

The FIQ is calling for measures intended to facilitate work-family balance, affecting, for example, overtime, in addition to a law on nurse/patient ratios in order to reduce their workload.

The Alliance du personnel professionnel et technique de la santé et des services sociaux (APTS) wants its members such as speech therapists, physiotherapists and nutritionists reimbursed for contributions to their professional orders, a cost it estimates at $25 million.

“It’s difficult when, for all the proposals we make, they reject them or they ask us to make choices between our members,” APTS first vice-president Josée Fréchette said. “We represent 108 job titles. Making choices and prioritizing members, we can’t do that.”

Article content
The Fédération de la santé et des services sociaux (FSSS-CSN) said Legault’s statements are counter-productive.

“The premier must stop adding fuel to the fire. For weeks, each statement from the premier has harmed the negotiations. He constantly distorts reality. Those who are inflexible at the table are the government. They have not wanted to move one iota on their priorities for months,” said FSSS president Réjean Leclerc.

Workers “want us to reduce their workload and find ways to attract and retain people in the network,” argues the FSSS.

Meanwhile, the health strike is “delaying 500 surgeries” per day, said Health Minister Christian Dubé.

Dubé said nurses would be willing to be moved from one hospital to another, “at their discretion,” but that the union opposed it.

He also maintained that to reduce compulsory overtime and the use of private placement agencies, “I need adjustments” from the unions, in particular recognizing the seniority of nurses who worked in the private sector and who would be ready to come to the public (sector).”
'''
montreal_gazette_5 = '''
Longueuil said Wednesday that its long-delayed cull of deer that have overrun a local park will finally go ahead in the fall of 2024.

The city of Longueuil plans to deploy certified crossbow hunters to kill almost all the deer in Michel-Chartrand Park, leaving about 15 of the more than 117 deer that were in the urban forest at last count.

Article content
Municipal officials have been trying to carry out the cull since 2020 but have faced strong backlash, including threats against the mayor. In 2017, there were just 32 deer in the park.

Article content
Jonathan Tabarah, an executive committee member who represents the district where the park is located, said the patience of many residents who live in the district has been pushed to the limit by delays that were out of the city’s control.

“I encourage you to ask the citizens of my district, as well as taking a 10-minute walk in the park and seeing the state of Michel-Chartrand park, which is not at all what it should be,” he told a news conference.

Animal rights organizations and activists fought the cull plan in court, but the challenge ended in October when Quebec’s Court of Appeal upheld a decision authorizing the crossbow hunt. The court found the cull falls within the city’s legal rights when it comes to environment, nuisances and health and safety.

“I’d say it’s a done deal,” Mayor Catherine Fournier said of the plan outlined Wednesday. The city will apply to get a new permit from the provincial government and put out tenders for an organization to carry out the cull.

The city has considered other options, including sterilization, birth control or even transporting the deer to a refuge, but ultimately experts concluded the only viable short-term solution was to kill the animals.

Montreal Gazette Headline News
Headline News
Get the latest headlines, breaking news and columns.

youremail@email.com
Sign Up
By signing up you consent to receive the above newsletter from Postmedia Network Inc.

Article content
Article content
The city has repeatedly said the white-tailed deer are contributing to road accidents, increasing the risk of Lyme disease, travelling to nearby residential areas in search of food in addition to stripping vegetation in parks.

“The whole ecological equilibrium of the park is at stake … so the city feels that it has to to do what needs to be done in order to ensure that the park will remain with us,” said Louis-Pascal Cyr, a spokesman for the city.

The park will be closed to the public next fall to complete the task. The meat will be donated to local food banks and charities ahead of Christmas. While the final cost to cull the dear is unclear, Fournier said the legal fight has already cost the city $375,000.

City officials are hopeful the cull will be a one-time operation and once the population is reduced, they will able to employ different methods to control it without hunting.

Opposition to the plan remains. Several dozen people protested outside Longueuil’s city hall on Wednesday, calling for the decision to be rescinded. Natalie Moseley, a member of the group, argued that other kinds of control measures should have been employed long ago and other options remain on the table.

“We could still move them out of the park today, tomorrow, if they would decide otherwise,” Moseley said. “We have everything in place right now to make this a very good transition for these deer.”

The situation has also created tension toward elected officials in the suburb, where Fournier has been under police protection since she received a threat in September. Longueuil police Insp. Gino Iannone said a second threat made last week against Fournier and Tabarah is under investigation.
'''
montreal_gazette_6 = '''
The Legault government is reportedly ready to significantly increase university tuition for students from other Canadian provinces — and lower rates for francophone students from France, Belgium, and maybe Switzerland.

Quebec on Wednesday was also mulling over a possible new measure: requiring 80 per cent of international students to reach intermediate French proficiency by graduation, according to La Presse.

Article content
Article content
McGill and Concordia universities say the tuition hike and the French-proficiency rules would be disastrous, driving away many students from other provinces and countries.

Citing unnamed sources, La Presse reported on what Premier François Legault’s cabinet is considering behind closed doors.

Most students from other provinces who come to Quebec to study do so at one of three English universities: Bishop’s, Concordia or McGill.

The government’s initial plan, announced in October, would have almost doubled the out-of-province rate, to $17,000, as of fall 2024. Now, Quebec is reportedly ready to set it at $12,000. That’s a 33-per-cent increase compared to current rates.

English universities and Quebec’s largest employer’s group, the Conseil du patronat, have said this will still be too high.

They say a major tuition hike will scare off most students from other provinces, leading to a catastrophic drop in revenue. Concordia and McGill have already imposed hiring freezes and warn they may have to cut jobs. Bishop’s, Quebec’s smallest English university, has said it might not survive.

Montreal Gazette Headline News
Headline News
Get the latest headlines, breaking news and columns.

youremail@email.com
Sign Up
By signing up you consent to receive the above newsletter from Postmedia Network Inc.

Article content
Article content
Hoping to avert the tuition increase, Bishop’s, Concordia and McGill have offered to improve the teaching of French to students from the rest of Canada.

Under their offer, at least 40 per cent of non-French-speaking students enrolled in undergraduate programs would reach intermediate proficiency in French by the time they graduate. The English universities said their francization efforts would be an alternative to the tuition hike.

Instead, the government plans to do both — increase tuition while obliging the universities to francize students, La Presse reported.

And, instead of 40 per cent, the province is reportedly considering an 80-per-cent target. The universities could face penalties if the target, to be in place by fall 2025, isn’t met.

This would be “disastrous” because many international students “have zero notion of French,” Fabrice Labeau, McGill’s deputy provost of student life and learning, told the Montreal Gazette.

For such a student to reach intermediate proficiency would require “240 hours of courses or something like 18 university credits – that’s more than a normal semester.”

Article content
Labeau cited as an example a German student considering McGill vs. the University of Toronto. They would likely balk at McGill because of the extra semester, he said.

“The fact that we speak French in Montreal used to be an asset for recruitment,” he said. “We’re making it a punishment now.”

He added: “We’re slamming the door on the Canadian students (with the tuition hike). Now, if we add on top of that, the French requirement, we’re slamming the door on international students.”

Concordia spokesperson Fiona Downey said an 80-per-cent French target is “inconceivable and unattainable.” That university has seen a 32-per-cent drop in applications from international students, compared to the same period last year.

In Quebec City, Higher Education Minister Pascale Déry told reporters the government is “going to come with an update very, very shortly.”

Government officials later told the Gazette that an announcement was expected on Thursday.

Déry was asked about complaints from McGill and Concordia that the government has given them the cold shoulder. Both institutions said Tuesday they haven’t been able to reach Déry since late November.

Article content
“We had a couple of discussions and I did speak to them obviously in those discussions,” Déry responded.

La Presse reported Quebec may cut tuition by two-thirds for university students from France and Belgium. They already pay much less than other international students when they study in Quebec.

The province also wants to negotiate a deal that would give francophone students from Switzerland the same tuition discount.

Students from France and Belgium currently pay the same tuition as those from the rest of Canada — $8,992 — when studying for bachelor’s degrees in Quebec. Students from Quebec pay $2,881.

Under the new scenario, French and Belgian students would pay $2,881, bringing the rate back down to where it was before 2015.

Just over 16,000 students from the two countries were studying in Quebec in 2021, a recent government report said. The vast majority are from France. Quebec had a total of 48,400 international students in 2021.

Legault has indicated he may exempt Bishop’s from the tuition hike. That university will have “no comment until we get the official announcement from the minister,” a Bishop’s spokesperson said.

Article content
A coalition of 40 anglophone groups denounced the tuition and French-proficiency changes.

“If a government were trying to devise a plan to starve Quebec’s English-language universities out of existence, it would look a lot like this,” said Eva Ludvig, president of the Quebec Community Groups Network.

“McGill is a jewel in Quebec’s crown — one of the world’s leading universities and consistently the best in Canada,” she said. “Instead of celebrating it and urging it on to greater success, the Legault government wants to punish it — all in the name of protecting and promoting the French language against what it perceives to be a threat in downtown Montreal.”

RELATED STORIES

Premier François Legault and Higher Education Minister Pascale Déry attend an event at the HEC Montréal business school on Friday, Sept. 15, 2023. Concordia and McGill say they have not spoken to Déry since late November.
Legault is giving English universities the cold shoulder: McGill and Concordia
McGill University principal Deep Saini, left, and Fabrice Labeau, the school's deputy provost of student life and learning, at a media briefing in Montreal Thursday December 7, 2023.
McGill won’t rule out moving some operations to another province

Legault has said the tuition hike will protect the French language and boost French universities. He has said non-French-speaking students from the rest of Canada are anglicizing Montreal.

But the QCGN argues the plan will not add revenue to the French university system. And it says the government has produced “no evidence whatsoever that the presence of McGill and Concordia students in the downtown core contributes to the anglicization of the metropolis.”
'''

In [183]:
test_article(montreal_gazette_1)

['The article is predicted as: Real',
 'Probabilities per class: [0.993652   0.00634797]']

In [184]:
test_article(montreal_gazette_2)

['The article is predicted as: Real',
 'Probabilities per class: [0.99770737 0.00229261]']

In [185]:
test_article(montreal_gazette_3)

['The article is predicted as: Real',
 'Probabilities per class: [0.9903341  0.00966589]']

In [186]:
test_article(montreal_gazette_4)

['The article is predicted as: Real',
 'Probabilities per class: [0.99875903 0.001241  ]']

In [187]:
test_article(montreal_gazette_5)

['The article is predicted as: Real',
 'Probabilities per class: [0.9961505  0.00384957]']

In [190]:
test_article(montreal_gazette_6)

['The article is predicted as: Real',
 'Probabilities per class: [0.99770737 0.00229261]']